# Setup

In particular this notebook requires an environment with the following custom dockerfile instructions:
```
RUN pip install mlflow==2.11.3 transformers datasets ipywidgets torch torchvision --upgrade

```

In [1]:
%matplotlib inline

In [2]:
import os

import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from matplotlib import pyplot as plt
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

import mlflow

## Huggingface login
NOTE: you need create a huggingface account if you don't already have one and use it to generate a token in the next cell

In [4]:
notebook_login()

In [5]:
!git config --global credential.helper store

In [6]:
train_dataset, test_dataset = load_dataset("imdb", split=["train", "test"])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

In [8]:
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
from mlflow import MlflowClient

client = MlflowClient()

model_name = "financial-news-sentiment-analysis"
registered_model = client.create_registered_model(model_name)
with mlflow.start_run() as run:
    model_info = mlflow.pytorch.log_model(model, "model")
    
    runs_uri = model_info.model_uri
    
    # Create a new model version of the RandomForestRegression model from this run
    desc = "Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))

/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/04/24 15:07:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: financial-news-sentiment-analysis, version 1


In [9]:
print(mv)

In [10]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
text = "I am very excited today."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

print(output)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [12]:
from transformers import pipeline
classification = pipeline('text-classification', model=model, tokenizer=tokenizer)

model_name = "financial-news-sentiment-analysis-classification"
registered_model = client.create_registered_model(model_name)
with mlflow.start_run() as run:
    model_info = mlflow.transformers.log_model(classification, "model")
    
    runs_uri = model_info.model_uri
    
    # Create a new model version of the RandomForestRegression model from this run
    desc = "Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis -- text classifier"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))


/tmp/ipykernel_207/60245261.py:7: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  model_info = mlflow.transformers.log_model(classification, "model")
2024/04/24 15:08:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: financial-news-sentiment-analysis-classification, version 1


In [13]:
print(mv)

In [14]:
classification(text)

[{'label': 'positive', 'score': 0.9922448992729187}]